In [1]:
import sys 
import os
sys.path.append(os.path.abspath("/home/yamba/Desktop/CovidMDP/src"))

In [2]:
os.chdir("../src/models")

In [3]:
from numpy.random import default_rng
import numpy as np
from tqdm import tqdm

In [4]:
from models.simulate_pandemic import init_infection, spread_infection, lambda_leak_expose, update_population

In [5]:
from actions import city_restrictions

In [6]:
policy='Unrestricted'
days=350
step_size=7
disable_tqdm=False
seed=None

In [7]:
prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

g_pickle = '../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle'

In [8]:
rng = default_rng(seed)
pop_matrix, adj_list = init_infection(g_pickle)
data = []
total_steps = int(np.ceil(days/step_size))

if isinstance(policy, str):
    policy = total_steps * [policy]

if len(policy) < total_steps:
    raise ValueError(f'len of policy should be at least {total_steps}')

for day in tqdm(range(1, days+1), disable=disable_tqdm):
    # if less than 90% already recovered, break simulation
    if (pop_matrix[pop_matrix[:, 1] == -1].shape[0] > pop_matrix.shape[0]*.9):
        break

    if day % step_size == 1:          
        current_step = int(day/step_size)
        restrictions = city_restrictions[policy[current_step]]

    pop_matrix = spread_infection(pop_matrix, adj_list, restrictions, day, rng, p_r)
    pop_matrix = lambda_leak_expose(pop_matrix, day)
    pop_matrix = update_population(pop_matrix)

    data.append(pop_matrix[:, 0:2])

100%|██████████| 350/350 [00:13<00:00, 25.72it/s]


In [10]:
import pandas as pd

In [13]:
pd.DataFrame([pd.Series(d[:, 1]).value_counts() for d in data])

,-1,0,1,2,3
0,NaN,55460.0,32.0,NaN,NaN
1,NaN,55458.0,29.0,5.0,NaN
2,NaN,55455.0,26.0,11.0,NaN
3,1.0,55451.0,28.0,12.0,NaN
4,2.0,55447.0,25.0,18.0,NaN
...,...,...,...,...,...
345,4996.0,50374.0,52.0,69.0,1.0
346,5009.0,50362.0,52.0,68.0,1.0
347,5023.0,50352.0,51.0,65.0,1.0
348,5037.0,50343.0,52.0,59.0,1.0


In [12]:
import networkx as nx

In [13]:
for x in p_r.items():
    print(x)

('home', 0.06)
('neighbor', 0.006)
('work', 0.006)
('school', 0.009)


In [14]:
G = nx.read_gpickle(g_pickle)


In [15]:
unique_types = np.unique([data['edge_type'] for x,y, data in G.edges(data=True)])

edge_list = {
    edge_type: np.array([(u,v) for u,v,e in G.edges(data=True) if e['edge_type'] == edge_type])
                            for edge_type in unique_types
}

In [28]:
currently_infected = rng.choice(G.nodes, size=10000)

In [35]:
%%timeit
exposed = []

def filter_contacts(rel, prob, restrictions, currently_infected, edge_list):
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])

    return contacts[mask]

args = [restrictions, currently_infected, edge_list]
exposed = list(map(lambda x: filter_contacts(x[0], x[1], *args), [x for x in p_r.items()]))
exposed = np.unique(np.concatenate(exposed)) 

33.3 ms ± 630 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
%%timeit
exposed = []

def filter_contacts(rel, prob, restrictions, currently_infected, edge_list):
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])

    return contacts[mask]

args = [restrictions, currently_infected, edge_list]
exposed = list(map(lambda x: filter_contacts(x[0], x[1], *args), [x for x in p_r.items()]))
exposed = np.unique(np.concatenate(exposed)) 

32.3 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%%timeit

exposed = []
for rel, prob in p_r.items():
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])
    exposed += [contacts[mask]]

exposed = np.unique(np.concatenate(exposed))

33.1 ms ± 494 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


array([], dtype=int64)

In [106]:
np.unique(np.concatenate(exposed)).shape

(1,)

In [70]:
selected_edges = [(u,v) for u,v,e in G.edges(data=True) if e['edge_type'] == 'school']
arr = np.array(selected_edges)

In [71]:
mask = np.isin(arr[:, 0], 

arr[mask][:,1].shape

(34341,)

In [54]:
arr[arr[:, 0].isin(rng.choice(G.nodes, size=10000))]

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [41]:
import scipy.sparse as sparse